In [40]:
import sys
import os
import lxml.etree as ET 
import seaborn as sns

In [94]:
def getDataFromFileRRT(filePath: str) -> tuple:
    tree = ET.parse(filePath)
    root = tree.getroot()
    time = float(root.find('time').text)
    # countOfEdges = int(root.find('countofedges').text)
    distance = float(root.find('distance').text)
    return (time, distance)
    

In [95]:
def getDataFromFileAstar(filePath: str) -> tuple:
    tree = ET.parse(filePath)
    root = tree.getroot()
    log = root.find('log')
    summary = log.find('summary')
    time = float(summary.get('time'))
    
    distance = float(summary.get('length'))
    return (time, distance)

